In [1]:
%matplotlib inline 
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.grid_search import GridSearchCV
from sklearn import cross_validation
from sklearn import grid_search

/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
from process_data import process_data

In [3]:
price_frame = process_data()
# print price_frame.head()

In [4]:
df_price_frame = price_frame.set_index("time")
df_price_frame.head()

,lmp_value,hour,day,month,summary,precipIntensity,precipProbability,temperature,apparentTemperature,dewPoint,humidity,pressure,windSpeed,windBearing,cloudCover,visibility
time,,,,,,,,,,,,,,,,
2016-01-01 07:00:00,30.56196,7,4,1,5,0.0,0.0,36.78,33.10,32.47,0.84,1020.18,4.53,111.0,0.00,9.69
2016-01-01 08:00:00,32.26382,8,4,1,5,0.0,0.0,39.36,34.56,33.89,0.81,1020.67,6.69,112.0,0.00,9.59
2016-01-01 09:00:00,30.84944,9,4,1,5,0.0,0.0,46.70,43.65,35.51,0.65,1021.00,6.16,119.0,0.01,9.61
2016-01-01 10:00:00,30.71027,10,4,1,5,0.0,0.0,52.42,52.42,35.37,0.52,1021.58,3.27,105.0,0.00,9.80
2016-01-01 11:00:00,28.87975,11,4,1,5,0.0,0.0,55.66,55.66,37.79,0.51,1021.44,1.78,339.0,0.17,9.92


In [18]:
train_mask = (df_price_frame.index > '2016-01-01') & (df_price_frame.index < '2016-03-01') & (df_price_frame["hour"] >= 0) 
test_mask = (df_price_frame.index > '2016-03-01') & (df_price_frame.index < '2016-03-5') & (df_price_frame["hour"] >= 0) #& (df_price_frame["lmp_value"] <= 30) 

price_train = df_price_frame.loc[train_mask].dropna()
price_test = df_price_frame.loc[test_mask].dropna()

XX_price_train = price_train.drop('lmp_value', axis = 1).reset_index().drop('time', axis = 1)
XX_price_test = price_test.drop('lmp_value', axis = 1).reset_index().drop('time', axis = 1)

YY_price_train = price_train['lmp_value']
YY_price_test = price_test['lmp_value']

In [ ]:
gamma_range = [0.01, 0.001, 0.0001, 0.00001]
epsilon_range = [x * 0.1 for x in range(0, 1)]
C_range = range(1, 500, 100)
tuned_parameters = [{
    'kernel': ['rbf'],
    'C': C_range,
    'gamma': gamma_range,
    'epsilon': epsilon_range,
#     'degree': degree_range
    }]

svr_price = GridSearchCV(SVR(), param_grid=tuned_parameters, verbose=0)
y_pred = svr_price.fit(XX_price_train, YY_price_train).predict(XX_price_test)

print 'Optimum parameters epsilon and kernel for SVR: ', svr_price.best_params_

print "The test score R2 for SVR: ", svr_price.score(XX_price_test, YY_price_test)

print("SVR mean squared error: %.2f"
      % np.mean((YY_price_test - svr_price.predict(XX_price_test)) ** 2))

In [ ]:
fig = plt.figure(figsize=(15,7))
plt.scatter(XX_price_test.index, YY_price_test, c='k', label='Observed')
plt.plot(XX_price_test.index, y_pred, c='r', label='Predicted')
plt.xlabel('data')
plt.ylabel('target')
plt.title('Support Vector Regression')
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(6,6))
plt.scatter(YY_price_test, YY_price_test, c='k')
plt.scatter(YY_price_test, y_pred, c='r')
plt.xlabel('Observed Elec. Usage (kWh)')
plt.ylabel("Predicted Elec. Usage (kWh): $\hat{Y}_i$")
plt.title("Energy vs Predicted Energy: $Y_i$ vs $\hat{Y}_i$")